In [12]:
import boto3
from botocore.exceptions import ClientError
import json
import psycopg2 as pg
import sqlite3

In [13]:
def get_pg_secret():
    secret_name = "rds!db-087d55ac-2a56-4890-84e2-077ddf7542c8"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    return json.loads(get_secret_value_response['SecretString'])

In [14]:
def copy_rows():
    split = lambda s: s.split(",") if s else []
    
    pg_secret = get_pg_secret()
    pg_conn = pg.connect(
        database="reports",
        user=pg_secret["username"],
        password=pg_secret["password"],
        host="reports.c786ks06yy45.us-east-2.rds.amazonaws.com",
        port=5432,
    )

    sqlite_conn = sqlite3.connect("reports.db")
    with sqlite_conn, pg_conn, pg_conn.cursor() as pg_cur, open("pg_schema.sql") as schema_f:
        # First create the schema
        pg_cur.execute(schema_f.read())
        
        count = 0
        for row in sqlite_conn.execute("SELECT * FROM report"):
            (report_id, publish_time, title, summary, upload_time,
             source, ipv4s, ipv6s, urls, yara_rules, cves,
             sha256s, md5s, sha1s, mitre, report_type,
             web_url, report_data_url) = row

            pg_row = {
                "id": report_id,
                "publish_time": publish_time,
                "title": title,
                "summary": summary,
                "upload_time": upload_time,
                "source": source,
                "ipv4s": split(ipv4s),
                "ipv6s": split(ipv6s),
                "urls": split(urls),
                "yara_rules": json.loads(yara_rules),
                "cves": split(cves),
                "sha256s": split(sha256s),
                "md5s": split(md5s),
                "sha1s": split(sha1s),
                "mitre": mitre,
                "report_type": report_type,
                "web_url": web_url,
                "report_data_url": report_data_url,
            }
            pg_cur.execute("""
                INSERT INTO report VALUES (
                    %(id)s,
                    to_timestamp(%(publish_time)s),
                    %(title)s,
                    %(summary)s,
                    to_timestamp(%(upload_time)s),
                    %(source)s,
                    %(ipv4s)s,
                    %(ipv6s)s,
                    %(urls)s,
                    %(yara_rules)s,
                    %(cves)s,
                    %(sha256s)s,
                    %(md5s)s,
                    %(sha1s)s,
                    %(mitre)s,
                    %(report_type)s,
                    %(web_url)s,
                    %(report_data_url)s
                );
            """, pg_row)

            count += 1

        print(f"Copied {count} rows")

    pg_conn.close()
    sqlite_conn.close()

In [15]:
if __name__ == '__main__':
    copy_rows()

Copied 305 rows
